# 1. 환경 설정 및 상품 정보 수집

In [ ]:
from selenium import webdriver 
from selenium.webdriver import ActionChains 
import time
from selenium.webdriver.common.by import By
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from tqdm import tqdm
import requests
import re

In [ ]:
## 선호
#냉장/냉동/간편요리
category_1 = 'https://www.coupang.com/np/categories/502492?listSize=120&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=1&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=&component=420166&rating=0'
#생수/음료/커피
category_2 = 'https://www.coupang.com/np/categories/503684?listSize=120&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=1&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=&component=420166&rating=0'

## 중간
#과일
category_3 = 'https://www.coupang.com/np/categories/420186?listSize=120&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=1&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=&component=420166&rating=0'
#채소
category_4 = 'https://www.coupang.com/np/categories/420208?listSize=120&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=1&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=&component=420166&rating=0'

## 비선호
#수산/건어물
category_5 = 'https://www.coupang.com/np/categories/420266?listSize=120&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=1&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=&component=420166&rating=0'
# 정육/계란
category_6 = 'https://www.coupang.com/np/categories/420234?listSize=120&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page=1&channel=user&fromComponent=N&selectedPlpKeepFilter=&sorter=bestAsc&filter=&component=420166&rating=0'

In [ ]:
def get_category_rank(url):
    custom_header = {'user-agent':'Mozilla','referer':url}
    
    product_dict = {'category' : [], 'product_name':[], 'product_price' : [], 'product_link' : []} 
    
    # headers 정보는 접속하는 서버를 속이기 위해서 사용한다
    data = requests.get(url, headers=custom_header)
    html = BeautifulSoup(data.text, "html.parser")
    products = html.select('.baby-product')
    title = html.select_one('.newcx-product-list-title').text.split()[0]
    
    for rank, product in enumerate(products, start=1): #1부터 시작해서 하나씩
        link = product.find("a", {"class": "baby-product-link"})
        url = link.get('href')
        url = 'https://www.coupang.com' + url
        
        is_rocket = product.select_one(".rocket")
        if is_rocket:
            p_name = product.select_one(".name")
            base_price = product.select_one('.base-price')
            sale_price = product.select_one('.price-value')
            real_price = base_price.text.strip() if base_price else sale_price.text.strip()

            product_dict['category'].append(title.strip())
            product_dict['product_name'].append(p_name.text.strip())
            product_dict['product_price'].append(real_price) # 가격 넣으려면 try, except or if문으로 처리
            product_dict['product_link'].append(url)
            
    prodcut_df = pd.DataFrame.from_dict(product_dict)
    return prodcut_df

# 사전에 정의한 1~6 링크 데이터 정보 수집
category_1_df = get_category_rank(category_1)
category_2_df = get_category_rank(category_2)
category_3_df = get_category_rank(category_3)
category_4_df = get_category_rank(category_4)
category_5_df = get_category_rank(category_5)
category_6_df = get_category_rank(category_6)


# 데이터프레임 생성
category_df = pd.concat([category_1_df, category_2_df, category_3_df, category_4_df, category_5_df, category_6_df], axis = 0)
category_df.reset_index(drop= True, inplace = True)
print(category_df.shape)
category_df.head()

(720, 4)


,category,product_name,product_price,product_link
0,냉장/냉동/간편요리,"곰곰 납작한 떡갈비 (냉동), 1000g, 1개","9,350",https://www.coupang.com/vp/products/1271810164...
1,냉장/냉동/간편요리,"곰곰 갈비탕, 600g, 5개","15,990",https://www.coupang.com/vp/products/187491448?...
2,냉장/냉동/간편요리,"곰곰 갈비탕, 600g, 4개","14,090",https://www.coupang.com/vp/products/187491448?...
3,냉장/냉동/간편요리,"하림 치킨너겟 (냉동), 1200g, 1개","12,570",https://www.coupang.com/vp/products/4567751809...
4,냉장/냉동/간편요리,"비비고 왕교자 (냉동), 1.4kg, 1개","10,410",https://www.coupang.com/vp/products/206820279?...


# 2. 상품 리뷰 페이지 수집

In [ ]:
# 상품 ID 추출
def func(str):
    return re.findall('\d+', str)[0]

category_df['product_id'] = list(map(func, category_df['product_link']))

# 리뷰 링크 생성
category_df['review_link'] ='https://www.coupang.com/vp/product/reviews?productId=' + category_df['product_id'] + '&page=1&size=100&sortBy=ORDER_SCORE_ASC&ratings&q&viRoleCode=3&ratingSummary=true'
print(category_df.shape)
category_df.head()

(720, 6)


,category,product_name,product_price,product_link,product_id,review_link
0,냉장/냉동/간편요리,"곰곰 납작한 떡갈비 (냉동), 1000g, 1개","9,350",https://www.coupang.com/vp/products/1271810164...,1271810164,https://www.coupang.com/vp/product/reviews?pro...
1,냉장/냉동/간편요리,"곰곰 갈비탕, 600g, 5개","15,990",https://www.coupang.com/vp/products/187491448?...,187491448,https://www.coupang.com/vp/product/reviews?pro...
2,냉장/냉동/간편요리,"곰곰 갈비탕, 600g, 4개","14,090",https://www.coupang.com/vp/products/187491448?...,187491448,https://www.coupang.com/vp/product/reviews?pro...
3,냉장/냉동/간편요리,"하림 치킨너겟 (냉동), 1200g, 1개","12,570",https://www.coupang.com/vp/products/4567751809...,4567751809,https://www.coupang.com/vp/product/reviews?pro...
4,냉장/냉동/간편요리,"비비고 왕교자 (냉동), 1.4kg, 1개","10,410",https://www.coupang.com/vp/products/206820279?...,206820279,https://www.coupang.com/vp/product/reviews?pro...


In [ ]:
product_dict = {'category' : [], 'product' : [], 'user' : [], 'date':[], 'rating' : [], 'headline' : [], 'review' : []} 
start = time.time()

for category_name, post_url in tqdm(zip(category_df['category'], category_df['review_link'])):
    
    # 1. 사이트 접속
    driver = webdriver.Chrome() 
    driver.set_window_size(1500, 800)
    driver.get(post_url)
    time.sleep(1)
    
    ## 크롤링
    # 1. html 수집
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    contents = soup.select('article')
    
    # 2. 100개 리뷰들에 대한 파싱

    for reviews in contents:
        # 카테고리 
        product_dict['category'].append(category_name)

        # 상품 명
        product = reviews.select_one('div.sdp-review__article__list__info__product-info__name').text.strip()
        product_dict['product'].append(product)

        # 유저
        user = reviews.select_one('div.sdp-review__article__list__info__user').text.strip()
        product_dict['user'].append(user)

        # 날짜
        date = reviews.select_one('div.sdp-review__article__list__info__product-info__reg-date').text.strip()
        product_dict['date'].append(date)

        # 평점
        rating = reviews.select_one('div.sdp-review__article__list__info__product-info__star-gray > div')['data-rating']
        product_dict['rating'].append(rating)

        # 제목
        try:
            headline = reviews.select_one('div.sdp-review__article__list__headline').text.strip()
            product_dict['headline'].append(headline)
        except:
            headline = 'NULL'
            product_dict['headline'].append(headline)

        try:
            # 리뷰
            review = reviews.select_one('div.sdp-review__article__list__review').text.strip()
            product_dict['review'].append(review)
        except:
            review = 'NULL'
            product_dict['review'].append(review)
        
    # 종료
    driver.close()
    
print('{}초가 소요되었습니다.'.format(time.time() - start))
final_df = pd.DataFrame.from_dict(product_dict)
final_df

720it [1:08:32,  5.71s/it]

4112.200327157974초가 소요되었습니다.


,category,product,user,date,rating,headline,review
0,냉장/냉동/간편요리,"곰곰 납작한 떡갈비 (냉동), 1000g, 1개",여자스윙스,2021.09.01,5,굿굿 떡갈비안좋아하는데 맛있어요 반찬으로 딱 ㅎㅎ,"광고아님) 솔직한 “내돈내산” 장/단점 갑니다. ❗❗\n♡♡♡도움이돼셨다면 ""도움이..."
1,냉장/냉동/간편요리,"곰곰 납작한 떡갈비 (냉동), 1000g, 1개",신*옥,2021.08.06,5,단짠단짠 맛있어요,곰곰 떡갈비 두번째 구매에요~~ \n\n첫번째구매때 넘맛있게 잘먹고 곰곰 고기 산적...
2,냉장/냉동/간편요리,"곰곰 납작한 떡갈비 (냉동), 1000g, 1개",퐁퐁팡팡,2021.08.06,5,떡갈비를 이용한 레시피 첨부합니다!!,"<곰곰 납작한 떡갈비, 냉동>\n중량 : 1,000g\n배송일자 : 2021.08...."
3,냉장/냉동/간편요리,"곰곰 납작한 떡갈비 (냉동), 1000g, 1개",하고하고,2021.07.19,5,저녁메뉴추천 곰곰 냉동떡갈비,가성비갑 밑반찬으로 먹을 고기류 뭐 없을까 하다가 곰곰 납작한 떡갈비 쿠팡 했다\n...
4,냉장/냉동/간편요리,"곰곰 납작한 떡갈비 (냉동), 1000g, 1개",주*옥,2021.07.11,4,NULL,☑️구매날짜 : 2021 / 06 /17\n\n☑️도착날짜 : 2021 /06 /1...
...,...,...,...,...,...,...,...
68781,정육/계란,"마니커 1등급 닭아랫날개 윙 (냉장), 600g, 1개",앗싸좋아,2020.04.17,1,왜 쓴 맛이.나는지요?? 제가 요리를 못한 건지 ㅡㅡ,제가 요리를 잘 못한 건지.모르겠는제요.\n알코올 넣고 요리하는데 엄창 써서 다시 ...
68782,정육/계란,"마니커 1등급 닭아랫날개 윙 (냉장), 600g, 1개",배송은담날,2021.02.04,5,가성비 좋은 닭윙,가격이 쿠팡프레시에서 상당히 저렴한편인\n닭윙입니다.\n두가지 요리 해요.\n절반은...
68783,정육/계란,"마니커 1등급 닭아랫날개 윙 (냉장), 1kg, 1개",멀킹,2021.05.23,4,냉동이랑 차이를 모르겠어요...,"더 촉촉 할 줄 알았는데, 냉동이랑 맛, 질감, 향 차이를 모르겠어요..."
68784,정육/계란,"마니커 1등급 닭아랫날개 윙 (냉장), 600g, 1개",May,2021.04.28,5,NULL,사실큰기대안했는데 와~~ 코땡트땡에서 산것보다 잡냄새도없이 맛나네요~ 이것저것먹어봐...


# 3. 데이터 수집 확인

In [ ]:
final_df['category'].value_counts()

냉장/냉동/간편요리    11881
채소            11872
정육/계란         11799
커피/음료/생수      11771
수산/건어물        10881
과일            10582
Name: category, dtype: int64

In [ ]:
# 리뷰가 없는 제거
final_df = final_df[final_df['review'] !='NULL']

# 중복된 리뷰 제거
final_df.drop_duplicates('review', inplace = True)
print(final_df.shape)

final_df['category'].value_counts()

(49048, 7)


<ipython-input-23-1b08a336bfff>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df.drop_duplicates('review', inplace = True)


냉장/냉동/간편요리    10132
채소             9213
정육/계란          8635
커피/음료/생수       7210
수산/건어물         7024
과일             6834
Name: category, dtype: int64

# 4. 저장

In [ ]:
# 1. 상품 정보 저장
category_df.to_csv('./data/cupang_product_info.csv', index = False, encoding = 'utf-8-sig')

# 2. 상품 리뷰 저장
final_df.to_csv('./data/cupang_review_df.csv', index =False, encoding = 'utf-8-sig')